In [ ]:
#creating spark instance
from pyspark.sql import SparkSession
spark = SparkSession .builder.appName('removeduplicatescart').getOrCreate()

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from pyspark.sql import functions as sf

In [ ]:
#Read column names
headers1 = spark.read.csv("sample1.csv",inferSchema =True, header=True)
#headers1.printSchema()

In [ ]:
#Read data from tsv file
rows = spark.read.option("sep", "\t").csv("..\hit_data.tsv",inferSchema =True, header=False)
#rows.printSchema()

In [ ]:
#adding header to data
rowsWithHeader1 = headers1.union(rows)
#rowsWithHeader1.printSchema()
rowsWithHeader1.count()

In [ ]:
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.exclude_hit<=0)
rowsCount = rowsWithHeader1.count()
rowsCount

In [ ]:
#filtering rows based on hit_source column value. Exclude all rows with hit_source = 5,7,8,9.
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=5)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=7)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=8)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=9)

In [ ]:
#Selecting only post columns
postData = rowsWithHeader1.select(*filter(lambda col: 'post_' in col,rowsWithHeader1.columns))
#Printing no. of columns
len(postData.columns)

In [ ]:
#Add a new column which conains values to idenify unique visitors
postDataWithUniqueId = postData.withColumn('joined_column', sf.concat(sf.col('post_visid_high'),sf.lit(''), sf.col('post_visid_low')))

In [ ]:
#apart from post_ columns considering column obtained after joining the two post_ columns as mentioned above and mcvisid
postDataWithUniqueId = postDataWithUniqueId.withColumnRenamed('joined_column', 'post_uniqueId')

In [ ]:
#Select the column post_uniqueID
UniqueId=postDataWithUniqueId.select('post_uniqueId')
#Print count
UniqueId.count()

In [ ]:
#Removing post_mobile columns from dataframe
postDataWithoutMobile = postDataWithUniqueId.drop(*filter(lambda col: 'mobile' in col,postData.columns))
#Printing no. of columns
len(postDataWithoutMobile.columns)

In [ ]:
#Get column names
#columnNames = postDataWithoutMobile.schema.names
#columnNames
#type(columnNames)

In [ ]:
#postDataWithoutMobile.select('post_referrer').show()

In [ ]:
#select users with cart in referrer
#cartRows = postDataWithoutMobile.filter(postDataWithoutMobile.post_referrer.rlike("^(.)*(cart|CART|Cart)(.)*$"))
#cartRows.count()

In [ ]:
#select users with cart,ship in referrer
#cartShipRows = postDataWithoutMobile.filter(postDataWithoutMobile.post_referrer.rlike("^(.)*(cart|CART|Cart|shipping|Ship|ship)(.)*$"))
#cartShipRows.count()

In [ ]:
#select users with cart,ship,chkout4_confirm in referrer
cartShipConfirmRows = postDataWithoutMobile.filter(postDataWithoutMobile.post_referrer.rlike("^(.)*(cart|CART|Cart|shipping|Ship|ship|chkout4_confirm)(.)*$"))
cartShipConfirmRows.count()

In [ ]:
#storing all existing distinct users
#uniqueIdsWithCart = [list(x.asDict().values())[0] for x in cartRows.select("post_uniqueid").distinct().collect()]
#print("{}".format(len(uniqueIdsWithCart)))

In [ ]:
#storing all existing distinct users
#uniqueIdsWithCartShip = [list(x.asDict().values())[0] for x in cartShipRows.select("post_uniqueid").distinct().collect()]
#print("{}".format(len(uniqueIdsWithCartShip)))

In [ ]:
#storing all existing distinct users
uniqueIdsWithCartShipConfirm = [list(x.asDict().values())[0] for x in cartShipConfirmRows.select("post_uniqueid").distinct().collect()]
print("{}".format(len(uniqueIdsWithCartShipConfirm)))

In [ ]:
#Convert to pandas dataframe
#cartPDF = cartRows.toPandas()
#cartPDF.shape

In [ ]:
#Removes any columns with null values
#cartRowsWithoutNullColumns = cartPDF.dropna(axis=1,how='all')
#cartRowsWithoutNullColumns.shape

In [ ]:
#List of column names after removing null columns
#cartRowsWithoutNullColumnsNames = list(cartRowsWithoutNullColumns)
#cartRowsWithoutNullColumnsNames

In [ ]:
#uniqueCartUsers = cartPDF.post_uniqueId.unique()
#len(uniqueCartUsers)

In [ ]:
#Drops duplicate columns and also drops null columns
#cartRowsWithoutDuplicateColumns = cartRowsWithoutNullColumns.T.drop_duplicates().T
#cartRowsWithoutDuplicateColumns.shape

In [ ]:
#List of column names after removing duplicate columns
#cartRowsWithoutDuplicateColumnsNames = list(cartRowsWithoutDuplicateColumns)
#cartRowsWithoutDuplicateColumnsNames

In [ ]:
#List of removed columns
#removedDuplicateColumns = list(set(cartRowsWithoutNullColumnsNames) - set(cartRowsWithoutDuplicateColumnsNames))
#removedDuplicateColumns

In [ ]:
#Save dataframe to a csv file
#cartRowsWithoutDuplicateColumns.to_csv("reduced.csv", index=False)

In [ ]:
#Get all rows of users having cart in post_referrer
#allRowsOfCartUsers = postDataWithoutMobile.where(postDataWithoutMobile.post_uniqueId.isin(uniqueIdsWithCart))
#Print count
#allRowsOfCartUsers.count()

In [ ]:
#Get all rows of users having cart,ship in post_referrer
#allRowsOfCartShipUsers = postDataWithoutMobile.where(postDataWithoutMobile.post_uniqueId.isin(uniqueIdsWithCartShip))
#Print count
#allRowsOfCartShipUsers.count()

In [ ]:
#Get all rows of users having cart, ship, chkout4_confirm in post_referrer
allRowsOfCartShipConfirmUsers = postDataWithoutMobile.where(postDataWithoutMobile.post_uniqueId.isin(uniqueIdsWithCartShipConfirm))
#Print count
allRowsOfCartShipConfirmUsersCount = allRowsOfCartShipConfirmUsers.count()
allRowsOfCartShipConfirmUsersCount

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
#event1CartRows = allRowsOfCartUsers.filter(allRowsOfCartUsers.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))
#print count
#event1CartRows.count()

In [ ]:
#Get unique Ids
#event1CartUniqueIds = [list(x.asDict().values())[0] for x in event1CartRows.select("post_uniqueid").distinct().collect()]
#print count
#print("{}".format(len(event1CartUniqueIds)))

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
#event1CartShipRows = allRowsOfCartShipUsers.filter(allRowsOfCartShipUsers.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))
#Print count
#event1CartShipRows.count()

In [ ]:
#Get unique Ids
#event1CartShipUniqueIds = [list(x.asDict().values())[0] for x in event1CartShipRows.select("post_uniqueid").distinct().collect()]
#print count
#print("{}".format(len(event1CartShipUniqueIds)))

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
event1CartShipConfirmRows = allRowsOfCartShipConfirmUsers.filter(allRowsOfCartShipConfirmUsers.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))
#print count
event1CartShipConfirmRows.count()

In [ ]:
#Get unique Ids
event1CartShipConfirmUniqueIds = [list(x.asDict().values())[0] for x in event1CartShipConfirmRows.select("post_uniqueid").distinct().collect()]
#print count
print("{}".format(len(event1CartShipConfirmUniqueIds)))

In [ ]:
#Get abandoned userIds
abandonedUsersIds = list(set(uniqueIdsWithCartShipConfirm) - set(event1CartShipConfirmUniqueIds))
#print count
print("{}".format(len(abandonedUsersIds)))

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
#event1PostRows = postDataWithoutMobile.filter(postDataWithoutMobile.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))
#print count
#event1PostRows.count()

In [ ]:
#Get unique Ids
#event1PostUniqueIds = [list(x.asDict().values())[0] for x in event1PostRows.select("post_uniqueid").distinct().collect()]
#print count
#print("{}".format(len(event1PostUniqueIds)))

In [ ]:
#event1Test = event1PostRows.select('post_uniqueid','post_referrer')
#event1Test.show()
#event1TestPDF = event1Test.toPandas()
#event1TestPDF.post_referrer

In [ ]:
#Save dataframe to a csv file
#event1TestPDF.to_csv("event1testpdf.csv", index=False)

In [ ]:
#Reducing data by removing null columns
allRowsOfCartShipConfirmUsersPDF = allRowsOfCartShipConfirmUsers.toPandas()
allRowsOfCartShipConfirmUsersPDF.shape

In [ ]:
#Removes any columns with null values
rowsWithoutNullColumns = allRowsOfCartShipConfirmUsersPDF.dropna(axis=1,how='all')
rowsWithoutNullColumns.shape

In [ ]:
#Removing columns with 50% nulls 
rowsWithoutNullColumnsThresh50 = rowsWithoutNullColumns.dropna(axis=1,thresh=allRowsOfCartShipConfirmUsersCount/2)
rowsWithoutNullColumnsThresh50.shape

In [ ]:
#Save dataframe to a csv file
rowsWithoutNullColumnsThresh50.to_csv("reducedusersrows.csv", index=False)

In [ ]:
#Read column names
allUsers = spark.read.csv("reducedusersrows.csv",inferSchema =True, header=True)
#allUsers.printSchema()

In [ ]:
#get all rows of abandoned users
allRowsOfAbandonedUsers = allUsers.where(allUsers.post_uniqueId.isin(abandonedUsersIds))
#print count
allRowsOfAbandonedUsers.count()

In [ ]:
#allRowsOfAbandonedUsersPDF = allRowsOfAbandonedUsers.toPandas()
#Removes any columns with null values
#allRowsOfAbandonedUsersPDF = allRowsOfCartShipConfirmUsersPDF.dropna(axis=1,how='all')
#allRowsOfAbandonedUsersPDF.shape

In [ ]:
#Groupby abandoned users using post_uniqueId
abandonedUsersGroupBy = allRowsOfAbandonedUsers.groupBy("post_uniqueId").count()

In [ ]:
abandonedUsersGroupBy.show()

In [ ]:
#get all rows of purchased users
allRowsOfPurchasedUsers = allUsers.where(allUsers.post_uniqueId.isin(event1CartShipConfirmUniqueIds))
#print count
allRowsOfPurchasedUsers.count()

In [ ]:
#Groupby purchased users using post_uniqueId
purchasedUsersGroupBy = allRowsOfPurchasedUsers.groupBy("post_uniqueId").count()

In [ ]:
purchasedUsersGroupBy.show()